실습 예제  
--------------
## 1.	data_concrete.csv 를 사용한 콩크리트의 강도 문제를 (선형회귀)를 TensorFlow로 풀으시오 (주의 Keras 아님!!!). strength ~ . 와 같은 수식을 적용하시오.  

#### a.	설명변수를 정규화하여 전처리 하시오.  
#### b.	3개의 노드로 이루어진 하나의 은닉층을 만드시오.  
#### c.	수치예측 (회귀)에 적합한 출력층 구조를 만드시오.  
#### d.	In-Sample 테스팅한 결과를 MSE와 RMSE로 정리하여 보시오.  





In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [2]:
data_origin = pd.read_csv("test_data\data_concrete.csv", engine='python')

In [3]:
data_origin.head()

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [24]:
data = data_origin.drop(["strength"], axis=1)
labels = data_origin["strength"].values

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 8 columns):
cement          1030 non-null float64
slag            1030 non-null float64
ash             1030 non-null float64
water           1030 non-null float64
superplastic    1030 non-null float64
coarseagg       1030 non-null float64
fineagg         1030 non-null float64
age             1030 non-null int64
dtypes: float64(7), int64(1)
memory usage: 64.5 KB


In [26]:
data.describe()

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000


원래는 트레인 데이터와 테스트 데이터를 나눠야 하지만 이번 문제에서는 생략한다.

## a.	설명변수를 정규화하여 전처리 하시오.  

In [29]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


std = StandardScaler()
data_prepared = std.fit_transform(data)

tmp = pd.DataFrame(data_prepared, columns=data.columns)
tmp.describe()

C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,cement,slag,ash,water,superplastic,coarseagg,fineagg,age
count,1.030000e+03,1.030000e+03,1.030000e+03,1.030000e+03,1.030000e+03,1.030000e+03,1.030000e+03,1.030000e+03
mean,-4.552992e-16,-1.241725e-16,-5.518779e-17,-1.655634e-16,-8.278168e-17,6.760504e-16,-4.759946e-16,2.069542e-17
std,1.000486e+00,1.000486e+00,1.000486e+00,1.000486e+00,1.000486e+00,1.000486e+00,1.000486e+00,1.000486e+00
min,-1.715253e+00,-8.568879e-01,-8.471439e-01,-2.800211e+00,-1.039143e+00,-2.212138e+00,-2.240917e+00,-7.073594e-01
25%,-8.500535e-01,-8.568879e-01,-8.471439e-01,-7.808939e-01,-1.039143e+00,-5.265174e-01,-5.319697e-01,-6.123314e-01
50%,-7.915193e-02,-6.017783e-01,-8.471439e-01,1.608294e-01,3.271508e-02,-6.329352e-02,7.386739e-02,-2.797331e-01
75%,6.589606e-01,8.007446e-01,1.002278e+00,4.887927e-01,6.691307e-01,7.267605e-01,6.291661e-01,1.637312e-01
max,2.477915e+00,3.310675e+00,2.281084e+00,3.065647e+00,4.353642e+00,2.214224e+00,2.733062e+00,5.057677e+00


## b.	3개의 노드로 이루어진 하나의 은닉층을 만드시오.  

In [80]:
labels.reshape(-1,1)

array([[79.99],
       [61.89],
       [40.27],
       ...,
       [23.7 ],
       [32.77],
       [32.4 ]])

In [51]:
n_inputs = 8
n_hidden1 = 3
n_outputs = 1

In [75]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')

In [76]:
hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", 
                          activation=tf.nn.relu)

## c.	수치예측 (회귀)에 적합한 출력층 구조를 만드시오.  

회귀에는 linear 활성화 함수가 적합하다고 생각한다.

In [77]:
output = tf.layers.dense(hidden1, n_outputs, name="outputs",
                         activation=None, use_bias=True)

#### d.	In-Sample 테스팅한 결과를 MSE와 RMSE로 정리하여 보시오.  

loss 함수와 optimizer함수를 정의하자

In [78]:
loss = tf.reduce_mean(tf.square(output-y))
loss2 = tf.sqrt(loss)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss)

train을 하자

In [81]:
init = tf.global_variables_initializer()

n_epochs = 1000

with tf.Session() as sess:
    init.run()
    for i in range(n_epochs):
        sess.run(train, feed_dict={X: data_prepared, y: labels.reshape(-1,1)})
    
    y_pred = output.eval(feed_dict={X: data_prepared})
    final_mse = mean_squared_error(labels, y_pred)
    final_rmse = np.sqrt(final_mse)
    print("final ---- mse, rmse = %s, %s" %(final_mse, final_rmse))
    print(y_pred.shape)

final ---- mse, rmse = 101.45789835386563, 10.072631153470558
(1030, 1)


In [83]:
display(pd.DataFrame(labels).describe(),columns=['labels'])
display(pd.DataFrame(y_pred, columns=['y_pred']).describe())

TypeError: publish() got an unexpected keyword argument 'columns'

기본 선형회귀의 rmse와 mse비교

In [16]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(data_prepared, labels)

train_predictions = lin_reg.predict(data_prepared)
lin_mse = mean_squared_error(labels, train_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_mse, lin_rmse

(107.19723607486019, 10.353609808895648)

In [17]:
display(pd.DataFrame(labels).describe())
display(pd.DataFrame(train_predictions, columns=['y_pred_linreg']).describe())

,strength
count,1030.000000
mean,35.817961
std,16.705742
min,2.330000
25%,23.710000
50%,34.445000
75%,46.135000
max,82.600000


,y_pred_linreg
count,1030.000000
mean,35.817961
std,13.106502
min,10.455766
25%,25.995461
50%,33.457700
75%,45.102870
max,78.990876


딥러닝 결과가 더 안좋은 이유를 모르겠다.

In [18]:
data_prepared.shape

(1030, 8)

In [19]:
labels.values.shape

(1030,)

In [20]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(3,input_dim=8, activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error',
                  optimizer='sgd')

model.fit(data_prepared, labels, epochs= 1000, batch_size=50)

y_predictions = model.predict(data_prepared).flatten()

Using TensorFlow backend.


Epoch 1/2000
1030/1030 [==============================] - 0s 221us/step - loss: 574.3814
Epoch 2/2000
1030/1030 [==============================] - 0s 73us/step - loss: 137.1253
Epoch 3/2000
1030/1030 [==============================] - 0s 71us/step - loss: 142.6081
Epoch 4/2000
1030/1030 [==============================] - 0s 68us/step - loss: 120.9082
Epoch 5/2000
1030/1030 [==============================] - 0s 66us/step - loss: 123.2057
Epoch 6/2000
1030/1030 [==============================] - 0s 65us/step - loss: 119.3038
Epoch 7/2000
1030/1030 [==============================] - 0s 68us/step - loss: 126.0499
Epoch 8/2000
1030/1030 [==============================] - 0s 67us/step - loss: 117.4998
Epoch 9/2000
1030/1030 [==============================] - 0s 66us/step - loss: 106.7758
Epoch 10/2000
1030/1030 [==============================] - 0s 66us/step - loss: 105.0929
Epoch 11/2000
1030/1030 [==============================] - 0s 62us/step - loss: 117.6018
Epoch 12/2000
1030/1030 [====

Epoch 94/2000
1030/1030 [==============================] - 0s 58us/step - loss: 52.5689
Epoch 95/2000
1030/1030 [==============================] - 0s 57us/step - loss: 51.8810
Epoch 96/2000
1030/1030 [==============================] - 0s 65us/step - loss: 53.2421
Epoch 97/2000
1030/1030 [==============================] - 0s 54us/step - loss: 68.8921
Epoch 98/2000
1030/1030 [==============================] - 0s 53us/step - loss: 50.5946
Epoch 99/2000
1030/1030 [==============================] - 0s 58us/step - loss: 58.4678
Epoch 100/2000
1030/1030 [==============================] - 0s 61us/step - loss: 55.2733
Epoch 101/2000
1030/1030 [==============================] - 0s 62us/step - loss: 58.5276
Epoch 102/2000
1030/1030 [==============================] - 0s 59us/step - loss: 69.0918
Epoch 103/2000
1030/1030 [==============================] - 0s 56us/step - loss: 53.5333
Epoch 104/2000
1030/1030 [==============================] - 0s 53us/step - loss: 50.8947
Epoch 105/2000
1030/1030 [=

1030/1030 [==============================] - 0s 67us/step - loss: 54.2416
Epoch 187/2000
1030/1030 [==============================] - 0s 89us/step - loss: 58.5475
Epoch 188/2000
1030/1030 [==============================] - 0s 78us/step - loss: 51.5731
Epoch 189/2000
1030/1030 [==============================] - 0s 56us/step - loss: 47.3403
Epoch 190/2000
1030/1030 [==============================] - 0s 57us/step - loss: 53.0427
Epoch 191/2000
1030/1030 [==============================] - 0s 57us/step - loss: 59.9032
Epoch 192/2000
1030/1030 [==============================] - 0s 56us/step - loss: 52.4486
Epoch 193/2000
1030/1030 [==============================] - 0s 58us/step - loss: 57.1684
Epoch 194/2000
1030/1030 [==============================] - 0s 58us/step - loss: 53.3740
Epoch 195/2000
1030/1030 [==============================] - 0s 57us/step - loss: 51.1041
Epoch 196/2000
1030/1030 [==============================] - 0s 57us/step - loss: 58.0492
Epoch 197/2000
1030/1030 [==========

1030/1030 [==============================] - 0s 75us/step - loss: 47.8419
Epoch 279/2000
1030/1030 [==============================] - 0s 72us/step - loss: 46.4121
Epoch 280/2000
1030/1030 [==============================] - 0s 62us/step - loss: 47.4418
Epoch 281/2000
1030/1030 [==============================] - 0s 58us/step - loss: 52.2284
Epoch 282/2000
1030/1030 [==============================] - 0s 57us/step - loss: 53.7386
Epoch 283/2000
1030/1030 [==============================] - 0s 59us/step - loss: 47.2789
Epoch 284/2000
1030/1030 [==============================] - 0s 70us/step - loss: 48.6107
Epoch 285/2000
1030/1030 [==============================] - 0s 75us/step - loss: 45.7115
Epoch 286/2000
1030/1030 [==============================] - 0s 60us/step - loss: 52.7461
Epoch 287/2000
1030/1030 [==============================] - 0s 63us/step - loss: 54.9878
Epoch 288/2000
1030/1030 [==============================] - 0s 53us/step - loss: 50.0832
Epoch 289/2000
1030/1030 [==========

...

1030/1030 [==============================] - 0s 62us/step - loss: 45.9155
Epoch 1560/2000
1030/1030 [==============================] - 0s 60us/step - loss: 47.3160
Epoch 1561/2000
1030/1030 [==============================] - 0s 56us/step - loss: 46.9692
Epoch 1562/2000
1030/1030 [==============================] - 0s 56us/step - loss: 44.9958
Epoch 1563/2000
1030/1030 [==============================] - 0s 57us/step - loss: 52.9019
Epoch 1564/2000
1030/1030 [==============================] - 0s 53us/step - loss: 45.8322
Epoch 1565/2000
1030/1030 [==============================] - 0s 56us/step - loss: 44.7481
Epoch 1566/2000
1030/1030 [==============================] - 0s 54us/step - loss: 49.8320
Epoch 1567/2000
1030/1030 [==============================] - 0s 52us/step - loss: 48.0578
Epoch 1568/2000
1030/1030 [==============================] - 0s 57us/step - loss: 46.7023
Epoch 1569/2000
1030/1030 [==============================] - 0s 57us/step - loss: 46.5878
Epoch 1570/2000
1030/1030 

1030/1030 [==============================] - 0s 80us/step - loss: 51.5838
Epoch 1651/2000
1030/1030 [==============================] - 0s 66us/step - loss: 45.6210
Epoch 1652/2000
1030/1030 [==============================] - 0s 60us/step - loss: 43.5362
Epoch 1653/2000
1030/1030 [==============================] - 0s 64us/step - loss: 45.9633
Epoch 1654/2000
1030/1030 [==============================] - 0s 84us/step - loss: 45.5586
Epoch 1655/2000
1030/1030 [==============================] - 0s 74us/step - loss: 47.9057
Epoch 1656/2000
1030/1030 [==============================] - 0s 65us/step - loss: 44.2994
Epoch 1657/2000
1030/1030 [==============================] - 0s 57us/step - loss: 48.5692
Epoch 1658/2000
1030/1030 [==============================] - 0s 58us/step - loss: 43.5343
Epoch 1659/2000
1030/1030 [==============================] - 0s 54us/step - loss: 45.9726
Epoch 1660/2000
1030/1030 [==============================] - 0s 63us/step - loss: 49.2778
Epoch 1661/2000
1030/1030 

1030/1030 [==============================] - 0s 59us/step - loss: 46.6598
Epoch 1742/2000
1030/1030 [==============================] - 0s 61us/step - loss: 46.7030
Epoch 1743/2000
1030/1030 [==============================] - 0s 58us/step - loss: 48.4366
Epoch 1744/2000
1030/1030 [==============================] - 0s 51us/step - loss: 47.2032
Epoch 1745/2000
1030/1030 [==============================] - 0s 58us/step - loss: 49.8217
Epoch 1746/2000
1030/1030 [==============================] - 0s 53us/step - loss: 44.0391
Epoch 1747/2000
1030/1030 [==============================] - 0s 60us/step - loss: 44.7227
Epoch 1748/2000
1030/1030 [==============================] - 0s 52us/step - loss: 44.0862
Epoch 1749/2000
1030/1030 [==============================] - 0s 59us/step - loss: 45.0490
Epoch 1750/2000
1030/1030 [==============================] - 0s 53us/step - loss: 47.9833
Epoch 1751/2000
1030/1030 [==============================] - 0s 57us/step - loss: 46.3700
Epoch 1752/2000
1030/1030 

Epoch 1832/2000
1030/1030 [==============================] - 0s 64us/step - loss: 46.5138
Epoch 1833/2000
1030/1030 [==============================] - 0s 58us/step - loss: 44.2236
Epoch 1834/2000
1030/1030 [==============================] - 0s 62us/step - loss: 47.0656
Epoch 1835/2000
1030/1030 [==============================] - 0s 61us/step - loss: 44.9132
Epoch 1836/2000
1030/1030 [==============================] - 0s 59us/step - loss: 47.7462
Epoch 1837/2000
1030/1030 [==============================] - 0s 65us/step - loss: 45.3945
Epoch 1838/2000
1030/1030 [==============================] - 0s 66us/step - loss: 50.5335
Epoch 1839/2000
1030/1030 [==============================] - 0s 56us/step - loss: 49.0334
Epoch 1840/2000
1030/1030 [==============================] - 0s 59us/step - loss: 46.6877
Epoch 1841/2000
1030/1030 [==============================] - 0s 58us/step - loss: 45.0450
Epoch 1842/2000
1030/1030 [==============================] - 0s 58us/step - loss: 44.6602
Epoch 1843

1030/1030 [==============================] - 0s 80us/step - loss: 46.9207
Epoch 1924/2000
1030/1030 [==============================] - 0s 79us/step - loss: 45.2525
Epoch 1925/2000
1030/1030 [==============================] - 0s 65us/step - loss: 45.9549
Epoch 1926/2000
1030/1030 [==============================] - 0s 64us/step - loss: 44.6129
Epoch 1927/2000
1030/1030 [==============================] - 0s 82us/step - loss: 49.6674
Epoch 1928/2000
1030/1030 [==============================] - 0s 76us/step - loss: 45.2863
Epoch 1929/2000
1030/1030 [==============================] - 0s 71us/step - loss: 50.0960
Epoch 1930/2000
1030/1030 [==============================] - 0s 67us/step - loss: 47.2801
Epoch 1931/2000
1030/1030 [==============================] - 0s 82us/step - loss: 50.4686
Epoch 1932/2000
1030/1030 [==============================] - 0s 75us/step - loss: 45.4954
Epoch 1933/2000
1030/1030 [==============================] - 0s 80us/step - loss: 49.1815
Epoch 1934/2000
1030/1030 

In [21]:
display(pd.DataFrame(labels).describe())
display(pd.DataFrame(y_predictions, columns=['y_pred_keras']).describe())

,strength
count,1030.000000
mean,35.817961
std,16.705742
min,2.330000
25%,23.710000
50%,34.445000
75%,46.135000
max,82.600000


,y_pred_keras
count,1030.000000
mean,35.374802
std,15.320550
min,0.508617
25%,24.845258
50%,35.116968
75%,44.994751
max,76.937767


## 2.	“data_spam.csv”를 가져오고 TensorFlow로 코딩하여 다음 물음에 답하여라. (설명 필수, 쥬피터 노트북 제출)   
### a.	데이터를 70% 학습용, 30% 시험용으로 분리 하여라.  
### b.	로지스틱 회귀 분류 예측 모형을 만들어서 학습 및 예측을 하여라.  
### c.	기준확률 = 0.5일 때 혼동행렬을 계산하여라.  
### d.	기준확률 = 0.5일 때 정확도, 민감도, 특이도, 정밀도를 계산하여라  

3.	순환신경망을 적용하여 문서 분류기를 만들어 보시오. 다음 순서를 참고하시오.  
a.	Keras패키지 사용. Embedding 모형 적용.  
b.	“\tweets”폴더 아래에 정리되어 있는 트윗을 읽어온다 (HINT: load_files() 사용).  
c.	텍스트를 전처리 한다.  
d.	A 또는 B 유형인 트윗을 분류예측하여 성능 (정확도)를 평가하시오 <= 70:30 학습/시험.  
e.	최대로 튜닝하여 90%이상의 정확도를 얻으시오.  